In [1]:
# Importing a few necessary libraries
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as pyplot

# Make matplotlib show our plots inline (nicely formatted in the notebook)
%matplotlib inline

# Load the dataset
data = pd.read_csv('trans.csv')

#data.describe
#[1654194 rows x 2 columns]

# Basic Data Description
print data.head()
#print data.shape
#print data.dtypes

            entry_time            exit_time
0  2012-12-01 07:26:00  2012-12-01 09:40:00
1  2012-12-01 07:36:00  2012-12-01 09:46:08
2  2012-12-01 07:41:00  2012-12-01 09:51:00
3  2012-12-01 06:57:00  2012-12-01 09:52:00
4  2012-12-01 07:04:00  2012-12-01 10:19:00


In [14]:
# Bad Data Check: when entry time >= exit time 

# Convert data from string to datetime
# entry = pd.to_datetime(data['entry_time'])
# exit = pd.to_datetime(data['exit_time'])

# n = len(data)
# def entry_lt_exit():
#     count=0
#     badrow=[]
#     for i in range(n):
#         if entry[i] >= exit[i]:
#             count+=1
#             badrow.attach(i)
#             #print i, entry[i], exit[i]
#     return count
# entry_lt_exit()


15

In [3]:
# Convert data from string to datetime
entry = pd.to_datetime(data['entry_time'])
exit = pd.to_datetime(data['exit_time'])

print entry.dtypes
print exit.dtypes

datetime64[ns]
datetime64[ns]


In [4]:
# Daily-Hourly Count 
# 
# dict()--- each date: 24 hours
# Every Transaction(entry vs exit) 3 Time States--- same date same hour, same date diff hour, diff date
# date: 24 hours--- count+1 when each transaction occupies this hour
# 
# Notice: After Running Below Once, Entry and Exit Set Changed
#
# This cell will run about 1 hr


from datetime import timedelta

hourly_data = dict()
    # {datetime.date(2012, 12, 1): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

n = len(data)
print n
for i in range(n):
    
    # Ignore Bad Data
    if entry[i] >= exit[i]:
        continue
    
    else:
        
        # Same Date Diff Hour, Diff Date
        while entry[i].date() < exit[i].date() or entry[i].hour < exit[i].hour:
            hourly_data[entry[i].date()] = hourly_data.get(entry[i].date(), [0] * 24)
                # [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            hourly_data[entry[i].date()][entry[i].hour] = hourly_data[entry[i].date()][entry[i].hour] + 1
            
            # Each Hour Occupied by Each Transaction
            entry[i] += timedelta(hours=1)
                # 2012-12-01 09:36:00, 1:00:00, 2012-12-01 10:36:00
        
        # Same Date Same Hour
        hourly_data[exit[i].date()] = hourly_data.get(exit[i].date(), [0] * 24)
        hourly_data[exit[i].date()][exit[i].hour] = hourly_data[exit[i].date()][exit[i].hour] + 1
        
    # Print % Data Processed   
    if i % 100000 == 0:
        print 1.0 * i / n * 100.0
#     if i==n:
#         print '100% Completed'

# Test i in range(5)--- Result Correct
# print data.head()
# print hourly_data
# {datetime.date(2012, 12, 1): [0, 0, 0, 0, 0, 0, 1, 5, 5, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
print i

1654194
0.0
6.04524015926
12.0904803185
18.1357204778
24.180960637
30.2262007963
36.2714409555
42.3166811148
48.361921274
54.4071614333
60.4524015926
66.4976417518
72.5428819111
78.5881220703
84.6333622296
90.6786023888
96.7238425481
1654193


In [24]:
len(hourly_data)

1157

In [21]:
hourly_data = dict((str(key), value) for (key, value) in hourly_data.items())
dict_slice=lambda one_dict,start,end: dict((k,one_dict[k]) for k in one_dict.keys()[start:end])
print dict_slice(hourly_data,0,2)
#2013-07-12 [9, 9, 10, 10, 14, 23, 53, 93, 135, 161, 168, 168, 156, 136, 109, 94, 76, 54, 38, 33, 29, 26, 21, 12]

{'2014-10-16': [56, 50, 48, 54, 81, 244, 907, 1709, 2346, 2521, 2587, 2623, 2573, 2560, 2507, 2415, 2121, 1387, 660, 291, 175, 128, 89, 71], '2014-06-30': [43, 41, 38, 39, 58, 195, 767, 1586, 2141, 2318, 2353, 2373, 2339, 2336, 2309, 2237, 2018, 1390, 680, 280, 150, 114, 98, 78]}


In [23]:
print hourly_data.keys()[:2]

['2014-06-30', '2014-10-16']


In [25]:
import json
with open('hourly_data.json','w') as fp:
    json.dump(hourly_data,fp)


In [2]:
#-------- Generate CSV file

import json

with open('hourly_data.json','rb') as f:
    hourly_data=f.readlines()
    # list,len=1

# Load hourly_data
hourly_data=json.loads(hourly_data[0])

In [6]:
len(hourly_data)

1157

In [16]:
import csv
with open('hourly_data.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    for key, value in hourly_data.items():
        spamwriter.writerow([key, value])

# To read it back:
# reader = csv.reader(open('hourly_data.csv', 'rb'))
# mydict = dict(reader)